In [ ]:
%pip install youtube-transcript-api openai pyonepassword

In [29]:
from youtube_transcript_api import YouTubeTranscriptApi
from openai import OpenAI
from pyonepassword import OP

op = OP()
openai_key = op.item_get('ociiuq6gchlgrjr2fopzbndrnq').credential
client = OpenAI(api_key=openai_key)

In [30]:
video_id = "g0CZVvenRa0" # ID from video URL
transcript = YouTubeTranscriptApi.get_transcript(video_id)
full_transcript = ' '.join(entry['text'] for entry in transcript)

In [31]:
chat_completion = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        { "role": "system", "content": "You summarize youtube transcripts to help traders determine a directional bias and have a profitable day." },
        {
            "role": "user",
            "content": f"Summarize the transcript and determine Tino's directional bias. Also read between the lines to determine what's actually being said and explain. Transcript: {full_transcript}",
        }
    ])
chat_completion.choices[0].message.content

"In this stream, Tino shares concerns about Bitcoin's short-term outlook. He highlights that there is a surge in open interest due to high leverage and low actual trading volume, which poses a risk for a price correction. Tino advises traders to take profits if they're in positive positions, as relying on the current upward movement could be risky without substantial trading volume backing it. There is also talk about external factors, such as the Bank of Japan’s interest rate decisions, which could indirectly affect Bitcoin and broader market dynamics.\n\nTino points out that Bitcoin has been moving up and testing higher levels, with a significant zone around $70,000. However, he warns that a pullback could occur, given the lack of substantial volume. He discusses external market factors including tech stocks, the U.S. dollar's strength, and potential implications of an ETF that could affect crypto markets. Importantly, he mentions that whales have been accumulating Bitcoin but warns 